In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from agenda_based_search import utils
from agenda_based_search.algorithms import bfs, dfs, ucs, ucs_improved, best_first_search

# Section 2

This notebook demonstrates the implementation of agenda-based search algorithms to find optimal train routes in the London tube system. All executable code related this section can be found in this notebook. Additionally, there are two more relavant files:
* `agenda_based_search/utils.py`: This contains the utility function `step_dictionary()` which is very similar to the helper code recommended in the coursework description.
* `agenda_based_search/algorithms.py`: All the variations of the search algorithms can be found here. 

In [3]:
df = pd.read_csv('tubedata.csv', 
                 names=['start','end','tube_line','average_time','zone1','zone2'],
                header=None)
df


,start,end,tube_line,average_time,zone1,zone2
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0
...,...,...,...,...,...,...
369,Victoria,Pimlico,Victoria,3,1,0
370,Pimlico,Vauxhall,Victoria,1,1,0
371,Vauxhall,Stockwell,Victoria,3,1,2
372,Stockwell,Brixton,Victoria,2,2,0


In [4]:
class TubeStationGraph:
    def __init__(self, df, reverse=False):
        self.station_dict, self.zone_dict, self.tube_line_dict = utils.step_dictionary(df)    # station_dict ----> {station: [(child_station, cost, tube_line)]}
                                                                                        # zone_dict ------> {station: {zone1, zone2}}
                                                                                        # tube_line_dict -----> {station: {tube_line1, tube_line2, ...}}
        self.station_names = list(self.station_dict.keys())                             # list of all station names
        if reverse:
            self.station_names = self.station_names[::-1]

        self.adj_matrix = self.create_adjacency_matrix()                                # adjacency matrix for cost between stations
        self.inverse_zone_dict = self.inverse_zone_dict()                               # inverse_zone_dict ----> {zone: {station1, station2, ...}}
        

    def create_adjacency_matrix(self):
        n = len(self.station_dict)
        adj_matrix = [[np.inf for _ in range(n)] for _ in range(n)]
        for station in self.station_names:
            for child, cost, _ in self.station_dict[station]:
                # Assumption: if there are multiple edges between two nodes, the cost is the minimum cost
                if cost < adj_matrix[self.station_names.index(station)][self.station_names.index(child)]:
                    adj_matrix[self.station_names.index(station)][self.station_names.index(child)]= cost
        return adj_matrix
    
    def inverse_zone_dict(self):
        inv_zone_dict = {}
        for station in self.zone_dict:
            for zone in self.zone_dict[station]:
                if zone not in inv_zone_dict:
                    inv_zone_dict[zone] = set()
                inv_zone_dict[zone].add(station)
        return inv_zone_dict
    

In [5]:
# Calculate total cost of the path
def total_cost(path, tube_graph):

    total_cost = 0
    for i in range(len(path)-1):
        total_cost += tube_graph.adj_matrix[tube_graph.station_names.index(path[i])][tube_graph.station_names.index(path[i+1])]
        print(f" {path[i]} to {path[i+1]} costs {tube_graph.adj_matrix[tube_graph.station_names.index(path[i])][tube_graph.station_names.index(path[i+1])]}")
        # print(f"Available lines between {path[i]} and {path[i+1]}: {tube_graph.station_dict[path[i]]}")
    print(f"Total cost without line changing penalty: {total_cost}")

    return total_cost


In [6]:
tube_graph = TubeStationGraph(df)

In [263]:
path, t, space = bfs('Mile End', 'New Cross Gate', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Mile End', 'Bethnal Green', 'Liverpool Street', 'Bank/Monument', 'London Bridge', 'Bermondsey', 'Canada Water', 'Surrey Quays', 'New Cross Gate']
Total cost: 20
Total nodes visited: 90
 Mile End to Bethnal Green costs 2
 Bethnal Green to Liverpool Street costs 3
 Liverpool Street to Bank/Monument costs 2
 Bank/Monument to London Bridge costs 2
 London Bridge to Bermondsey costs 3
 Bermondsey to Canada Water costs 2
 Canada Water to Surrey Quays costs 2
 Surrey Quays to New Cross Gate costs 4


20

In [265]:
path, t, space = dfs('Mile End', 'New Cross Gate', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Mile End', 'Bow Road', 'Bromley-by-Bow', 'West Ham', 'Canning Town', 'North Greenwich', 'Canary Wharf', 'Canada Water', 'Surrey Quays', 'New Cross Gate']
Total cost: 23
Total nodes visited: 237
 Mile End to Bow Road costs 1
 Bow Road to Bromley-by-Bow costs 2
 Bromley-by-Bow to West Ham costs 2
 West Ham to Canning Town costs 3
 Canning Town to North Greenwich costs 3
 North Greenwich to Canary Wharf costs 3
 Canary Wharf to Canada Water costs 3
 Canada Water to Surrey Quays costs 2
 Surrey Quays to New Cross Gate costs 4


23

In [266]:
path, t, space = ucs('Mile End', 'New Cross Gate', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Mile End', 'Stepney Green', 'Whitechapel', 'Shadwell', 'Wapping', 'Rotherhithe', 'Canada Water', 'Surrey Quays', 'New Cross Gate']
Total cost: 16
Total nodes visited: 66
 Mile End to Stepney Green costs 2
 Stepney Green to Whitechapel costs 3
 Whitechapel to Shadwell costs 2
 Shadwell to Wapping costs 1
 Wapping to Rotherhithe costs 1
 Rotherhithe to Canada Water costs 1
 Canada Water to Surrey Quays costs 2
 Surrey Quays to New Cross Gate costs 4


16

In [28]:
path, t, space = ucs_improved('Hammersmith', 'Heathrow Terminal 4', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, tube_graph)

Path: ['Hammersmith', 'Turnham Green', 'Acton Town', 'South Ealing', 'Northfields', 'Boston Manor', 'Osterley', 'Hounslow East', 'Hounslow Central', 'Hounslow West', 'Hatton Cross', 'Heathrow Terminals 1-2-3', 'Heathrow Terminal 4']
Total cost: 34
Total nodes visited: 168
 Hammersmith to Turnham Green costs 3
 Turnham Green to Acton Town costs 3
 Acton Town to South Ealing costs 4
 South Ealing to Northfields costs 1
 Northfields to Boston Manor costs 2
 Boston Manor to Osterley costs 3
 Osterley to Hounslow East costs 2
 Hounslow East to Hounslow Central costs 2
 Hounslow Central to Hounslow West costs 2
 Hounslow West to Hatton Cross costs 4
 Hatton Cross to Heathrow Terminals 1-2-3 costs 3
 Heathrow Terminals 1-2-3 to Heathrow Terminal 4 costs 5
Total cost without line changing penalty: 34


In [11]:
def zone_distance(tube_graph, n_samples=10, seed=42):
    '''
    Calculate distance between zones by sampling 10 random stations from each zone and doing breadth first search between them
    '''
    np.random.seed(seed)
    zone_names = list(tube_graph.inverse_zone_dict.keys())
    zone_distance_matrix = [[0 for _ in range(len(zone_names))] for _ in range(len(zone_names))]
    for i in range(len(zone_names)):
        for j in range(i+1, len(zone_names)):
            total_distance = 0
            for _ in range(n_samples):
                start = np.random.choice(list(tube_graph.inverse_zone_dict[zone_names[i]]))
                end = np.random.choice(list(tube_graph.inverse_zone_dict[zone_names[j]]))
                path, _, _ = bfs(start, end, tube_graph)
                total_distance += len(path)
            zone_distance_matrix[i][j] = total_distance / n_samples
            zone_distance_matrix[j][i] = total_distance / n_samples

    return zone_distance_matrix, zone_names

zone_distance_matrix, zone_names = zone_distance(tube_graph)
print(f"Zone list: {zone_names}")
zone_distance_matrix

Zone list: ['5', '4', '3', '2', '1', '6', 'a', 'b', 'c', 'd']


[[0, 19.8, 20.1, 16.8, 15.2, 20.2, 18.6, 22.8, 23.7, 26.4],
 [19.8, 0, 16.8, 17.1, 13.7, 16.8, 18.4, 21.9, 26.1, 22.0],
 [20.1, 16.8, 0, 12.6, 11.9, 24.0, 19.0, 22.7, 22.4, 25.0],
 [16.8, 17.1, 12.6, 0, 8.7, 19.7, 19.9, 20.0, 20.4, 21.5],
 [15.2, 13.7, 11.9, 8.7, 0, 16.9, 15.8, 18.1, 17.2, 20.4],
 [20.2, 16.8, 24.0, 19.7, 16.9, 0, 21.7, 21.1, 22.9, 23.3],
 [18.6, 18.4, 19.0, 19.9, 15.8, 21.7, 0, 3.9, 5.3, 4.9],
 [22.8, 21.9, 22.7, 20.0, 18.1, 21.1, 3.9, 0, 2.0, 2.8],
 [23.7, 26.1, 22.4, 20.4, 17.2, 22.9, 5.3, 2.0, 0, 2.0],
 [26.4, 22.0, 25.0, 21.5, 20.4, 23.3, 4.9, 2.8, 2.0, 0]]

In [7]:
# Best-first Search using zone distance heuristic

def heuristic_fn(node, end, tube_graph, zone_distance_matrix, zone_names):
    node_zone_set = tube_graph.zone_dict[node]
    end_zone_set = tube_graph.zone_dict[end]
    # Get the zone with the minimum distance from the end zone
    if node_zone_set.intersection(end_zone_set):
        return 0
    else:
        d = np.inf
        for e in end_zone_set:
            for n in node_zone_set:
                if d > zone_distance_matrix[zone_names.index(n)][zone_names.index(e)]: 
                    d = zone_distance_matrix[zone_names.index(n)][zone_names.index(e)]
        
    return d




In [19]:
import random
while True:
    start = random.choice(tube_graph.station_names)
    end = random.choice(tube_graph.station_names)
    path, t, space = best_first_search(start, end, tube_graph, zone_distance_matrix, zone_names)
    if len(path) > 15 or len(path) == 0:
        continue

    assert (total_cost(path, tube_graph) - t) % 2 == 0

 Manor House to Finsbury Park costs 2
 Finsbury Park to Highbury & Islington costs 2
 Highbury & Islington to King's Cross St. Pancras costs 4
 King's Cross St. Pancras to Euston costs 2
 Euston to Mornington Crescent costs 2
 Mornington Crescent to Camden Town costs 1
 Camden Town to Kentish Town costs 2
 Kentish Town to Tufnell Park costs 2
 Tufnell Park to Archway costs 2
 Archway to Highgate costs 3
Total cost without line changing penalty: 22
 Kingsbury to Wembley Park costs 4
 Wembley Park to Finchley Road costs 7
 Finchley Road to Baker Street costs 6
 Baker Street to Bond Street costs 2
 Bond Street to Green Park costs 2
 Green Park to Westminster costs 3
 Westminster to Waterloo costs 2
 Waterloo to Lambeth North costs 1
 Lambeth North to Elephant & Castle costs 3
Total cost without line changing penalty: 30
 Eastcote to Ruislip Manor costs 2
 Ruislip Manor to Ruislip costs 2
 Ruislip to Ickenham costs 3
Total cost without line changing penalty: 7
 Westminster to Green Park co

AssertionError: 

In [1]:
start = 'Vauxhall'
end = "Earls' Court"
end = "West Kensington"
path, t, space = ucs_improved(start, end, tube_graph)
# path, t, space = best_first_search(start, end, tube_graph, zone_distance_matrix, zone_names)


print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, tube_graph)

for i, route in enumerate(path[:-1]):
    s = path[i]
    e = path[i+1]
    row = df[(df['start'] == s) & (df['end'] == e) | (df['start'] == e) & (df['end'] == s)]
    row = row.to_dict('records')
    if len(row) > 1:
        print(f"-----------")
        for r in row:
            print(f"Between {r['start']} and {r['end']} == Takes {r['average_time']} on {r['tube_line']} line")

NameError: name 'ucs_improved' is not defined

In [90]:
def ucs_improved(start, end, tube_graph, change_time=2):

    # The state requires a modification to include th last line used
    queue = [(start, [start], 0, None, [])]        # state = (node, path traversed, cost, last line used)
    visited = set()
    while queue:
        # Get node from queue
        (node, path, cost, last_line_type, line_list) = queue.pop(0)
        if node not in visited:
            if node == end:    
            # Goal Test successful
                time_taken = cost           
                expanded_nodes = len(visited)
                print(f"Line list: {line_list}")
                return path, time_taken, expanded_nodes
            visited.add(node)
        
            # Expand node
            for child, child_cost, line in tube_graph.station_dict[node]:
                if last_line_type is None or last_line_type == line:
                    line_change_cost = 0
                else: 
                    line_change_cost = change_time

                if  path[-1] == "Gloucester Road" and child == "Earls' Court":
                    print(path[-1], child, cost, child_cost, line_change_cost, line)
                if path[-1] == "Earls' Court" and child == "West Kensington":
                    print(path[-1], child, cost, child_cost, line_change_cost, line)
                    
                queue.append((child, path + [child], cost + child_cost + line_change_cost, line, line_list + [line]))
            if path[-1] == "Gloucester Road":
                
            queue.sort(key=lambda x: x[2])          # Sort queue by cost

        
    return [], -1, len(visited)          # Retun failure